In [ ]:
import comtypes.client

def connect_to_etabs():
    try:
        helper = comtypes.client.CreateObject('ETABSv1.Helper')
        helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
        etabs_object = helper.GetObject("CSI.ETABS.API.ETABSObject") 
        return etabs_object.SapModel
    except Exception as e:
        print(f"Connection Error: {e}")
        return None

sap_model = connect_to_etabs()

In [9]:
import pandas as pd

# --- BASE SHEAR SCALING CHECK (TBDY 2018) ---
def get_base_shear_final(sap_model, load_case):
    """
    Retrieves the maximum base reaction (Base Shear) for a specific load case.
    Units are set to kN and m (Unit index 6).
    """
    # Set units to kN, m, C
    sap_model.SetPresentUnits(6) 
    
    # Selection of specific load cases for output to optimize performance
    sap_model.Results.Setup.DeselectAllCasesAndCombosForOutput()
    ret = sap_model.Results.Setup.SetCaseSelectedForOutput(load_case)

    if ret != 0:
        return 0.0, 0.0

    # Retrieve base reactions
    results = sap_model.Results.BaseReact()
    
    if not results or results[0] == 0:
        return 0.0, 0.0

    # result[4] contains Fx list, result[5] contains Fy list
    fx_list = results[4]
    fy_list = results[5]
    
    # Calculate absolute maximum values
    max_fx = max([abs(x) for x in fx_list])
    max_fy = max([abs(y) for y in fy_list])
    
    return max_fx, max_fy

def run_base_shear_check(sap_model):
    """
    Compares Equivalent Static (VtE) vs Response Spectrum (VtB) base shears.
    Applies the 0.9 beta factor as per TBDY 2018.
    """
    # Configuration for load case pairs (Equivalent Static vs Dynamic)
    # Ensure these names match your ETABS model exactly.
    load_pairs = [
        {"Dir": "X", "Static": "Ex", "Dynamic": "EX_RP"},
        {"Dir": "Y", "Static": "Ey", "Dynamic": "EY_RP"}
    ]
    BETA = 0.90
    report_data = []

    print(f"--- TBDY 2018 BASE SHEAR SCALING CHECK (Results in kN) ---")
    
    for p in load_pairs:
        # Fetch results for both cases
        res_static = get_base_shear_final(sap_model, p["Static"])
        res_dynamic = get_base_shear_final(sap_model, p["Dynamic"])

        # Assign values based on the direction being analyzed
        vt_e = res_static[0] if p["Dir"] == "X" else res_static[1]
        vt_b = res_dynamic[0] if p["Dir"] == "X" else res_dynamic[1]
        
        limit = BETA * vt_e
        
        scaling_factor = "-"
        status = "✅ PASS"

        print(f" > Analyzing Direction {p['Dir']}: Static={vt_e:.2f} kN, Dynamic={vt_b:.2f} kN")

        if vt_b == 0:
             status = "⚠️ ERROR (ZERO)"
             scaling_factor = "Check Load Case Names"
        elif vt_b < limit:
            # Calculate the required scaling factor if dynamic shear is insufficient
            factor = limit / vt_b
            status = "❌ INSUFFICIENT"
            scaling_factor = f"{factor:.2f}"

        report_data.append({
            "Direction": p["Dir"],
            "VtE (kN)": f"{vt_e:.2f}",
            "VtB (kN)": f"{vt_b:.2f}",
            "Limit (0.9*VtE)": f"{limit:.2f}",
            "Status": status,
            "Scaling Factor": scaling_factor
        })
    
    print("-" * 75)
    df_report = pd.DataFrame(report_data)
    print(df_report.to_string(index=False))
    return df_report

# --- EXECUTION ---
if 'sap_model' in locals() and sap_model is not None:
    final_report = run_base_shear_check(sap_model)
else:
    print("API Connection not detected. Please run the connection cell first.")

--- TBDY 2018 BASE SHEAR SCALING CHECK (Results in kN) ---
 > Analyzing Direction X: Static=4179.34 kN, Dynamic=413.43 kN
 > Analyzing Direction Y: Static=2954.82 kN, Dynamic=336.21 kN
---------------------------------------------------------------------------
Direction VtE (kN) VtB (kN) Limit (0.9*VtE)         Status Scaling Factor
        X  4179.34   413.43         3761.41 ❌ INSUFFICIENT           9.10
        Y  2954.82   336.21         2659.34 ❌ INSUFFICIENT           7.91
